## Job Web-Scraping
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [235]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en import stop_words

import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import coherencemodel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [236]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [237]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [238]:
#adds the data in URL format by removing any white space and adding the data. 
def Keyword_to_url(kw):
    kw = kw.strip(" ").replace(" ","%20")
    return kw

In [239]:
# this will be used as the URL input for multiple job searches. 
occupation = 'data analyst' #str(input("Job you are looking for: "))
location = 'austin, texas' #str(input("Where do you want to look: "))
radius = str("5")
page_sort = "sortcolumns=accquisitiondate&sortdirections=DSC"
page_size = "100" #how many pages would you like to get. 
source = 'IN' #stands for indeed

In [240]:
#give it the siteD
url = 'https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword='+ \
    Keyword_to_url(occupation)+\
    '&ajax=0&location='+location+'&radius='+radius+ \
    '&source='+ source +'&pagesize='+page_size+'&sortcolumns=accquisitiondate&sortdirections=DSC'
browser.get(url)
time.sleep(1)

In [241]:
#these catch the title of the job details. 
company = []
position = []
location = []
date_posted = []
pos_description = []

In [242]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-title="Job Title"]'))

In [243]:
listing_num = 15

In [244]:
job_links = browser.find_elements_by_xpath('//*[@data-title="Job Title"]')

In [245]:
success = 0
check =  []

In [246]:
range(listing_num-1)

range(0, 14)

In [248]:
browser.maximize_window()
browser.

In [249]:
for job in tqdm(range(listing_num-1)):
    #Retrieve the informaiton from the job table. 
    position.append(job_links[job].text)
    company.append(browser.find_elements_by_xpath('//*[@data-title="Company"]')[job].text)
    location.append(browser.find_elements_by_xpath('//*[@data-title="Location"]')[job].text)
    date_posted.append(browser.find_elements_by_xpath('//*[@data-title="Date Posted"]')[job].text)
    time.sleep(np.random.randint(1,20))

    #Changes the window to the new tab.
    try:
        job_links[job].click()
        browser.switch_to_window(browser.window_handles[-1])
        while len(browser.window_handles) < 2:
            time.sleep(np.random.randint(2,7))
            job_links[job].click()
        browser.switch_to_window(browser.window_handles[1])
        pos_description.append((browser.find_element_by_xpath('//*[@class="snip"]').text).replace("\n", " "))

        #close the window and open original window
        browser.close()
        browser.switch_to_window(browser.window_handles[0])
        success += 1
    except:
        check.append(job)
        pass
    


 43%|███████████████████████████████████▌                                               | 6/14 [01:58<02:37, 19.74s/it]

CannotSendRequest: Request-sent

In [252]:
len(company), len(position) , len(location), len(date_posted), len(pos_description)

(13, 13, 13, 13, 9)

In [253]:
pos_description

["Job Title: Digital Analytics Consultant, Decision Analytics (Risk Analsyt/Modeler) Location: US-TX-Austin Overview: Digital Analytics Consultant, Decision Analytics (Risk Analyst/Modeler) Responsibilities: The position is located at client’s site in US. Support strategic efforts within Analytics team in US and help manage a team of smart analysts in India Structure risk analytics problems and formulate analytical solutions: Credit Card Risk Modeling, Risk strategy, reporting, Fraud Analytics Deliver high intensity projects in individual capacity Support offshore coordination – Business Analyst Role -- Understanding requirement from the client team and aligning with India team for delivery Responsible for adherence to timelines and quality of output Qualifications: Master’s or Bachelor's degree in electronics/electrical engineering, computer science, IT or related quantitative analytics field from top-tier universities with strong record of achievement. 6+ years’ experience with solid

In [254]:
data.head()

,company,date_posted,location,position,position_description
0,Charles Schwab,06/21/2018,"Austin, TX",Operations Specialist,"Description: Austin - TX, AUS2, 2309 Gracy Far..."
1,Cray,06/21/2018,"Austin, TX",Business Analyst - Product Strategy,Who We Are Our business is supercomputing. We...
2,EXL,06/21/2018,"Austin, TX","Digital Analytics Consultant, Decision Analytics","Overview Digital Analytics Consultant, Decisio..."


In [255]:
browser.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
#data.to_csv("./data/"+str(datetime.datetime.today())[:10]+"job_list.csv")

## Using SpaCy for NLP - this is a Work in Progress
Objective: Setup for topic modeling and use LDA to determine feature importance

In [265]:
parser = spacy.lang.en.English()
nlp = spacy.load('en')

In [266]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(pos_description),deacc=True))

In [267]:
data_words = list(sent_to_words(pos_description))

In [268]:
data_words

[['job',
  'title',
  'digital',
  'analytics',
  'consultant',
  'decision',
  'analytics',
  'risk',
  'analsyt',
  'modeler',
  'location',
  'us',
  'tx',
  'austin',
  'overview',
  'digital',
  'analytics',
  'consultant',
  'decision',
  'analytics',
  'risk',
  'analyst',
  'modeler',
  'the',
  'position',
  'is',
  'located',
  'at',
  'client',
  'site',
  'in',
  'us',
  'support',
  'strategic',
  'efforts',
  'within',
  'analytics',
  'team',
  'in',
  'us',
  'and',
  'help',
  'manage',
  'team',
  'of',
  'smart',
  'analysts',
  'in',
  'india',
  'structure',
  'risk',
  'analytics',
  'problems',
  'and',
  'formulate',
  'analytical',
  'solutions',
  'credit',
  'card',
  'risk',
  'modeling',
  'risk',
  'strategy',
  'reporting',
  'fraud',
  'analytics',
  'deliver',
  'high',
  'intensity',
  'projects',
  'in',
  'individual',
  'capacity',
  'support',
  'offshore',
  'coordination',
  'business',
  'analyst',
  'role',
  'understanding',
  'requirement',
 

In [269]:
#Building bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [270]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [271]:
print(trigram_mod[bigram_mod[data_words[0]]])

['job', 'title', 'digital', 'analytics', 'consultant', 'decision', 'analytics', 'risk', 'analsyt', 'modeler', 'location', 'us', 'tx', 'austin', 'overview', 'digital', 'analytics', 'consultant', 'decision', 'analytics', 'risk', 'analyst', 'modeler', 'the', 'position', 'is', 'located', 'at', 'client', 'site', 'in', 'us', 'support', 'strategic', 'efforts', 'within', 'analytics', 'team', 'in', 'us', 'and', 'help', 'manage', 'team', 'of', 'smart', 'analysts', 'in', 'india', 'structure', 'risk', 'analytics', 'problems', 'and', 'formulate', 'analytical', 'solutions', 'credit', 'card', 'risk', 'modeling', 'risk', 'strategy', 'reporting', 'fraud', 'analytics', 'deliver', 'high', 'intensity', 'projects', 'in', 'individual', 'capacity', 'support', 'offshore', 'coordination', 'business', 'analyst', 'role', 'understanding', 'requirement', 'from', 'the', 'client', 'team', 'and', 'aligning', 'with', 'india', 'team', 'for', 'delivery', 'responsible', 'for', 'adherence', 'to', 'timelines', 'and', 'qual

In [272]:
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOP_WORDS] for doc in text]

def make_bigrams(text):
    return [bigram_mod[doc] for doc in text]

def make_trigrams(text):
    return [trigram_mod[bigram_mod[doc]] for doc in text]

def lemmatization(text, allowed_postags = ['NOUN','ADJ','VERB','ADV']):
    text_rem = []
    for sent in text:
        doc = nlp(" ".join(sent))
        text_rem.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return text_rem 

In [273]:
words_no_stops = remove_stopwords(data_words)

word_bigrams = make_bigrams(words_no_stops)

data_lemma =  lemmatization(word_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [274]:
data_lemma

[['job',
  'title',
  'digital',
  'analytic',
  'consultant',
  'decision',
  'analytic',
  'risk',
  'analsyt',
  'modeler',
  'location',
  'austin',
  'overview',
  'digital',
  'analytic',
  'consultant',
  'decision',
  'analytic',
  'risk',
  'analyst',
  'modeler',
  'position',
  'locate',
  'client',
  'site',
  'support',
  'strategic',
  'effort',
  'analytic',
  'team',
  'help',
  'manage',
  'team',
  'smart',
  'analyst',
  'india',
  'structure',
  'risk',
  'analytic',
  'problem',
  'formulate',
  'analytical',
  'solution',
  'credit',
  'card',
  'risk',
  'model',
  'risk',
  'strategy',
  'report',
  'fraud',
  'analytic',
  'deliver',
  'high',
  'intensity',
  'project',
  'individual',
  'capacity',
  'support',
  'offshore',
  'coordination',
  'business',
  'analyst',
  'role',
  'understanding',
  'requirement',
  'client',
  'team',
  'align',
  'india',
  'team',
  'delivery',
  'responsible',
  'adherence',
  'timelin',
  'quality',
  'output',
  'qualif

In [275]:
data_lemma

[['job',
  'title',
  'digital',
  'analytic',
  'consultant',
  'decision',
  'analytic',
  'risk',
  'analsyt',
  'modeler',
  'location',
  'austin',
  'overview',
  'digital',
  'analytic',
  'consultant',
  'decision',
  'analytic',
  'risk',
  'analyst',
  'modeler',
  'position',
  'locate',
  'client',
  'site',
  'support',
  'strategic',
  'effort',
  'analytic',
  'team',
  'help',
  'manage',
  'team',
  'smart',
  'analyst',
  'india',
  'structure',
  'risk',
  'analytic',
  'problem',
  'formulate',
  'analytical',
  'solution',
  'credit',
  'card',
  'risk',
  'model',
  'risk',
  'strategy',
  'report',
  'fraud',
  'analytic',
  'deliver',
  'high',
  'intensity',
  'project',
  'individual',
  'capacity',
  'support',
  'offshore',
  'coordination',
  'business',
  'analyst',
  'role',
  'understanding',
  'requirement',
  'client',
  'team',
  'align',
  'india',
  'team',
  'delivery',
  'responsible',
  'adherence',
  'timelin',
  'quality',
  'output',
  'qualif

In [276]:
id2word = corpora.Dictionary(data_lemma)

In [277]:
texts = data_lemma

In [278]:
corpus = [id2word.doc2bow(text) for text in texts]

In [279]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,num_topics=20,
                               random_state=42,
                               update_every=1,
                               chunksize=100,
                               passes=10,
                               alpha='auto',
                               per_word_topics=True)

In [280]:
doc_lda = lda_model[corpus]

In [281]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = coherencemodel.CoherenceModel(model=lda_model, texts=data_lemma, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.492321838234574

Coherence Score:  0.29621327832583


In [282]:
pyLDAvis.gensim.prepare(lda_model,corpus, id2word)

PreparedData(topic_coordinates=            Freq  cluster  topics         x             y
topic                                                    
14     99.988922        1       1 -0.085166  2.094037e-07
6       0.000763        1       2 -0.026747 -3.565294e-06
16      0.000573        1       3  0.001334  4.512648e-06
3       0.000573        1       4  0.001641 -2.989005e-06
11      0.000573        1       5  0.001587  9.153287e-05
0       0.000573        1       6  0.002275 -2.305765e-05
15      0.000573        1       7  0.002192  6.207841e-05
1       0.000573        1       8  0.003154  1.885399e-07
10      0.000573        1       9  0.004502 -3.286855e-05
17      0.000573        1      10  0.004838 -7.390215e-05
2       0.000573        1      11  0.006774  2.718586e-06
18      0.000573        1      12  0.007010 -2.360790e-05
4       0.000573        1      13  0.007071 -3.932550e-05
7       0.000573        1      14  0.008268 -2.113395e-05
13      0.000573        1      15  0.008161 -5.128628e-05
8       0.000572        1      16  0.009577 -2.154354e-05
19      0.000572        1      17  0.010459  6.038914e-05
12      0.000572        1      18  0.010529  9.060266e-06
9       0.000572        1      19  0.010831  4.299281e-05
5       0.000572        1      20  0.011709  1.959715e-05, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
377   Default  104.000000     analytic  104.000000  30.0000  30.0000
314   Default   70.000000       austin   70.000000  29.0000  29.0000
306   Default   61.000000     business   61.000000  28.0000  28.0000
341   Default   50.000000          ago   50.000000  27.0000  27.0000
481   Default   49.000000      analyst   49.000000  26.0000  26.0000
630   Default   48.000000          job   48.000000  25.0000  25.0000
138   Default   44.000000          day   44.000000  24.0000  24.0000
126   Default   44.000000       client   44.000000  23.0000  23.0000
204   Default   43.000000         risk   43.000000  22.0000  22.0000
470   Default   34.000000   experience   34.000000  21.0000  21.0000
598   Default   34.000000          exl   34.000000  20.0000  20.0000
180   Default   34.000000         team   34.000000  19.0000  19.0000
142   Default   32.000000         work   32.000000  18.0000  18.0000
617   Default   29.000000     employee   29.000000  17.0000  17.0000
293   Default   28.000000      process   28.000000  16.0000  16.0000
183   Default   27.000000      company   27.000000  15.0000  15.0000
343   Default   25.000000        apply   25.000000  14.0000  14.0000
615   Default   24.000000      service   24.000000  13.0000  13.0000
609   Default   23.000000    operation   23.000000  12.0000  12.0000
492   Default   22.000000   analytical   22.000000  11.0000  11.0000
591   Default   21.000000      product   21.000000  10.0000  10.0000
238   Default   21.000000      digital   21.000000   9.0000   9.0000
15    Default   20.000000        skill   20.000000   8.0000   8.0000
305   Default   20.000000   consultant   20.000000   7.0000   7.0000
43    Default   20.000000        datum   20.000000   6.0000   6.0000
70    Default   19.000000       schwab   19.000000   5.0000   5.0000
451   Default   19.000000      support   19.000000   4.0000   4.0000
52    Default   18.000000  opportunity   18.000000   3.0000   3.0000
3     Default   18.000000     decision   18.000000   2.0000   2.0000
216   Default   17.000000        drive   17.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
630   Topic20    0.000037          job   48.660431  -2.0159  -6.3122
138   Topic20    0.000036          day   44.532429  -1.9526  -6.3375
617   Topic20    0.000035     employee   29.802099  -1.5730  -6.3596
341   Topic20    0.000036          ago   50.580112  -2.0861  -6.3437
204   Topic20    0.000036         risk   43.623726  -1.9492  -6.3547
470   Topic20    0.000035   experience   34.712322  -1.7293